<a href="https://colab.research.google.com/github/Aljawharah004/Bank-Analysis/blob/main/datadrip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df_transaction = pd.read_csv('transactions_data.csv')
import matplotlib.pyplot as plt
import seaborn as sns

## MCC API

In [ ]:
!pip install requests pandas mastercard-oauth1-signer

### Making MCC fetch resilient

In [ ]:
import time, json, os, pandas as pd, requests
from tenacity import retry, wait_exponential_jitter, stop_after_attempt
import oauth1.authenticationutils as authenticationutils
from oauth1.oauth_ext import OAuth1RSA

# ==== CONFIG ====
CONSUMER_KEY = "OOPKSjljmCim6R4XEytMtGH8AYRu-wh3GMl1mmnh4285d91f!4cf35e5c1c404a1aaf934d99fb0bdf1f0000000000000000"  # from Mastercard Developer portal
P12_PATH     = "444200750-sandbox.p12"   # the file you uploaded
P12_PASSWORD = "9v3ThM5M4ggmF!RPfGSfHG6v7y!JC"                # set when you downloaded the key
BASE_URL     = "https://sandbox.api.mastercard.com/places"
OUT_PATH     = "mcc_lookup_mastercard.csv"
CACHE_PATH   = "mcc_pages_cache.jsonl"  # page-by-page cache

# OAuth
signing_key = authenticationutils.load_signing_key(P12_PATH, P12_PASSWORD)
oauth = OAuth1RSA(CONSUMER_KEY, signing_key)

# Retry for transient 5xx (503 etc.)
@retry(wait=wait_exponential_jitter(initial=1, max=20), stop=stop_after_attempt(6))
def fetch_page(limit: int, offset: int):
    url = f"{BASE_URL}/merchant-category-codes"
    params = {
        "limit": limit,
        "offset": offset,
        # if backend is touchy, try removing sort entirely:
        # "sort": "+merchantCategoryName"
    }
    r = requests.get(url, params=params, auth=oauth, headers={"Accept": "application/json"}, timeout=30)
    # Raise on HTTP errors (503 included) so tenacity retries
    r.raise_for_status()
    return r.json().get("merchantCategoryCodes", [])

def save_page_cache(items):
    with open(CACHE_PATH, "a", encoding="utf-8") as f:
        for it in items:
            f.write(json.dumps(it, ensure_ascii=False) + "\n")

def load_cache_df():
    if not os.path.exists(CACHE_PATH):
        return pd.DataFrame()
    rows = [json.loads(line) for line in open(CACHE_PATH, "r", encoding="utf-8")]
    return pd.DataFrame(rows)

# Try to resume from cache if any
df_cached = load_cache_df()
seen_count = len(df_cached)
print(f"Cached rows found: {seen_count}")

rows_total = []
if seen_count:
    rows_total.extend(df_cached.to_dict("records"))

# Page through with conservative settings (smaller pages help under load)
limit = 100
offset = 0 if not seen_count else (seen_count // limit) * limit

while True:
    try:
        items = fetch_page(limit, offset)
    except Exception as e:
        # last-ditch: small sleep and one manual retry for this offset
        time.sleep(3)
        try:
            items = fetch_page(limit, offset)
        except Exception as e2:
            print(f"Stopping at offset={offset} due to repeated errors: {e2}")
            break

    if not items:
        break

    save_page_cache(items)
    rows_total.extend(items)

    print(f"Fetched {len(items)} at offset {offset} (total so far: {len(rows_total)})")
    if len(items) < limit:
        break
    offset += limit

# Build DataFrame from all rows (cache + new)
mcc_df = pd.DataFrame(rows_total)
if mcc_df.empty:
    raise RuntimeError("No MCC rows fetched; try again in a few minutes.")

# Normalize column names (adjust if your JSON uses different keys)
mcc_df = mcc_df.rename(columns={
    "merchantCategoryCode": "mcc",
    "merchantCategoryName": "mcc_name",
    "industryCode": "industry_code",
    "industryName": "industry_name"
})
mcc_df["mcc"] = mcc_df["mcc"].astype(str).str.zfill(4)
mcc_df = mcc_df.drop_duplicates(subset=["mcc"])

mcc_df.to_csv(OUT_PATH, index=False)
print(f"Saved {len(mcc_df)} MCC rows to {OUT_PATH}")

Cached rows found: 0
Stopping at offset=0 due to repeated errors: RetryError[<Future at 0x7acd1789de90 state=finished raised HTTPError>]


RuntimeError: No MCC rows fetched; try again in a few minutes.

### Mapping MCC names

In [ ]:
# 1) Load transactions & MCC lookup
tx = pd.read_csv("transactions.csv")  # or read_excel(...)
mcc_lookup = pd.read_csv("mcc_lookup_mastercard.csv", dtype={"mcc": str})

# 2) Normalize MCC in tx and merge
tx["mcc_str"] = tx["mcc"].astype(str).str.extract(r"(\d{4})")[0].str.zfill(4)
tx_enriched = tx.merge(mcc_lookup, left_on="mcc_str", right_on="mcc", how="left")

# 3) Bucket for cleaner visuals
def bucketize(name):
    n = (name or "").lower()
    if any(k in n for k in ["grocery", "supermarket", "convenience"]): return "Groceries"
    if any(k in n for k in ["restaurant", "dining", "fast food", "cafe"]): return "Food & Dining"
    if any(k in n for k in ["hotel", "motel", "lodging"]): return "Travel - Lodging"
    if any(k in n for k in ["airline", "airport", "taxi", "transport"]): return "Travel - Transport"
    if any(k in n for k in ["fuel", "gas", "petrol"]): return "Fuel"
    if any(k in n for k in ["clothing", "apparel"]): return "Apparel"
    return "Other"

tx_enriched["mcc_bucket"] = tx_enriched["mcc_name"].apply(bucketize)

tx_enriched.to_csv("transactions_with_mcc_names.csv", index=False)
print("Saved transactions_with_mcc_names.csv")


#### Emergency fallback

In [ ]:
# Build a quick local dictionary for the most common MCCs you see (extend as needed)
minimal_map = {
    "5411": "Grocery Stores, Supermarkets",
    "5812": "Eating Places, Restaurants",
    "5814": "Fast Food Restaurants",
    "5541": "Service Stations",
    "5542": "Automated Fuel Dispensers",
    "4111": "Local/Suburban Commuter Transportation",
    "4511": "Airlines, Air Carriers",
    "5311": "Department Stores",
    "5699": "Misc. Apparel & Accessory Stores",
    "5732": "Electronics Stores",
    "5999": "Misc. & Specialty Retail",
    "6011": "ATM Financial Institutions",
    "6012": "Financial Institutions–Merchandise/Services",
    "4814": "Telecommunication Services",
    "4829": "Money Orders—Wire Transfer",
    "4900": "Utilities",
    "7011": "Lodging—Hotels, Motels, Resorts",
    "4112": "Passenger Railways",
    "4121": "Taxis/Limousines",
    "4789": "Transportation Services (Not Elsewhere Classified)"
}

tx = pd.read_csv("transactions.csv")
tx["mcc_str"] = tx["mcc"].astype(str).str.extract(r"(\d{4})")[0].str.zfill(4)
tx["mcc_name"] = tx["mcc_str"].map(minimal_map).fillna("Unknown")
tx.to_csv("transactions_with_mcc_names_partial.csv", index=False)
print("Saved transactions_with_mcc_names_partial.csv (temporary mapping)")


## general info about cards_data.csv


In [13]:
import pandas as pd

df = pd.read_csv("cards_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


,0
id,0
client_id,0
card_brand,0
card_type,0
card_number,0
expires,0
cvv,0
has_chip,0
num_cards_issued,0
credit_limit,0


In [14]:
# Check basic info
print("Shape:", df.shape)  # (rows, columns)
print(df.head())  # First 5 rows
print(df.columns)  # Column names

Shape: (6146, 13)
     id  client_id  card_brand        card_type       card_number  expires  \
0  4524        825        Visa            Debit  4344676511950444  12/2022   
1  2731        825        Visa            Debit  4956965974959986  12/2020   
2  3701        825        Visa            Debit  4582313478255491  02/2024   
3    42        825        Visa           Credit  4879494103069057  08/2024   
4  4659        825  Mastercard  Debit (Prepaid)  5722874738736011  03/2009   

   cvv has_chip  num_cards_issued credit_limit acct_open_date  \
0  623      YES                 2       $24295        09/2002   
1  393      YES                 2       $21968        04/2014   
2  719      YES                 2       $46414        07/2003   
3  693       NO                 1       $12400        01/2003   
4   75      YES                 1          $28        09/2008   

   year_pin_last_changed card_on_dark_web  
0                   2008               No  
1                   2014          

### cards_data.csv errors check


In [19]:
import pandas as pd

df = pd.read_csv("cards_data.csv")

# Remove $ and convert to numeric (NaN if invalid)
df["credit_limit"] = pd.to_numeric(df["credit_limit"].str.replace("$", "", regex=False), errors="coerce")

# Find invalid rows
invalid_rows = df[df["credit_limit"].isna() | (df["credit_limit"] < 0)]

print(invalid_rows)

Empty DataFrame
Columns: [id, client_id, card_brand, card_type, card_number, expires, cvv, has_chip, num_cards_issued, credit_limit, acct_open_date, year_pin_last_changed, card_on_dark_web]
Index: []


## general info about users_data.csv


In [6]:
import pandas as pd

df = pd.read_csv("users_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


,0
id,0
current_age,0
retirement_age,0
birth_year,0
birth_month,0
gender,0
address,0
latitude,0
longitude,0
per_capita_income,0


### userss_data.csv errors check

In [8]:
# Remove $ and commas, handle parentheses as negative values
for col in ["per_capita_income", "yearly_income", "total_debt"]:
    df[col] = (
        df[col]
        .astype(str)  # ensure string
        .str.replace("$", "", regex=False)  # remove dollar sign
        .str.replace(",", "", regex=False)  # remove commas
        .str.replace(r"\((.*?)\)", r"-\1", regex=True)  # convert (123) to -123
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")  # convert to numeric

## general info about transactions_data.csv

In [20]:
import pandas as pd

df = pd.read_csv("transactions_data.csv")

# General structure
df.info()

# Quick statistics
df.describe(include="all")

# Missing values count
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6912260 entries, 0 to 6912259
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       float64
 3   card_id         float64
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     float64
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             float64
 11  errors          object 
dtypes: float64(5), int64(1), object(6)
memory usage: 632.8+ MB


,0
id,0
date,0
client_id,1
card_id,1
amount,1
use_chip,1
merchant_id,1
merchant_city,1
merchant_state,781676
zip,825942


In [23]:
# Drop rows where any of the specified columns are null
df_clean = df.dropna(subset=["client_id", "card_id", "amount", "use_chip", "mcc"])

# Check the result
print(f"Rows after cleaning: {len(df_clean)}")

Rows after cleaning: 6912259


### transactions_data.csv errors check

In [24]:
# Clean amount column
df_clean["amount"] = (
    df_clean["amount"]
    .astype(str)  # ensure it's string
    .str.replace("$", "", regex=False)  # remove dollar sign
    .str.replace(",", "", regex=False)  # remove commas if any
    .str.replace(r"\((.*?)\)", r"-\1", regex=True)  # convert (123.45) to -123.45
)

# Convert to numeric
df_clean["amount"] = pd.to_numeric(df_clean["amount"], errors="coerce")

# Check
print(df_clean["amount"].head(20))

/tmp/ipython-input-1361183022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["amount"] = (


0     -77.00
1      14.57
2      80.00
3     200.00
4      46.41
5       4.81
6      77.00
7      26.46
8     261.58
9      10.74
10      3.51
11      2.58
12     39.63
13     43.33
14     49.42
15      1.09
16     73.79
17    100.00
18     26.04
19    -64.00
Name: amount, dtype: float64


/tmp/ipython-input-1361183022.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["amount"] = pd.to_numeric(df_clean["amount"], errors="coerce")
